In [2]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [3]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
def parse_result(result) -> MarketDataInfo:
    return MarketDataInfo(
        id=result["id"],
        symbol=result["symbol"],
        current_price=result["current_price"],
        market_cap=result["market_cap"],
        image=result["image"]
    )

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [14]:
#save as raw json
with open('zxc_historical_prices.json', 'w') as f:
    json.dump(a, f)

In [10]:
def get_historical_data(id: str) -> List[str]:
    return json.loads(
            call_get(API_ENDPOINT / "coins" / id / "market_chart", {"vs_currency" : "usd", "days" : "max"})
        )

In [11]:
a = get_historical_data('bitcoin')


,prices,market_caps,total_volumes
0,"[[1367107200000, 135.3], [1367193600000, 141.9...","[[1367107200000, 1500517590], [1367193600000, ...","[[1367107200000, 0], [1367193600000, 0.0], [13..."
